# Testing Models Individually


In [4]:
import tarfile
from scipy.io import loadmat
import pandas as pd
import os
import shutil
import torch
from torchvision.datasets import ImageFolder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.optim import lr_scheduler
import numpy as np
import time
import copy
import matplotlib.pyplot as plt

In [6]:
from torchvision.datasets import ImageFolder
test_folder = '102flowers_segmen_split/test'


# Define transformations for the test dataset
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for ResNet
])


# Load the test dataset
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder, transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the device for the testing script
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## VGG16

In [7]:
# Load the model
vgg_16 = models.vgg16(pretrained=False)
num_features = vgg_16.classifier[6].in_features
vgg_16.classifier[6] = nn.Linear(num_features, 102)
vgg_16.load_state_dict(torch.load('model_vgg16(8).pth'))

# Move the model to the device
vgg_16 = vgg_16.to(device)

# Set the model to evaluation mode
vgg_16.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = vgg_16(inputs)
        _, preds = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

vgg16_accuracy = correct / total * 100
print(f'Accuracy of the network on the test images: {vgg16_accuracy:.4f}')



c:\Users\alici\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Accuracy of the network on the test images: 73.2314


In [20]:
# Load the model
vgg_19 = models.vgg19(pretrained=False)
num_features = vgg_19.classifier[6].in_features
vgg_19.classifier[6] = nn.Linear(num_features, 102)
vgg_19.load_state_dict(torch.load('model_vgg19(13).pth'))

# Move the model to the device
vgg_19 = vgg_19.to(device)

# Set the model to evaluation mode
vgg_19.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = vgg_19(inputs)
        _, preds = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

vgg19_accuracy = correct / total * 100
print(f'Accuracy of the network on the test images: {vgg19_accuracy:.4f}')

Accuracy of the network on the test images: 71.0522


In [12]:
# Load the model
resnet_18 = models.resnet18(pretrained=False)
num_features = resnet_18.fc.in_features
resnet_18.fc = nn.Linear(num_features, 102)
resnet_18.load_state_dict(torch.load('resnet_18(5).pth'))

# Move the model to the device
resnet_18 = resnet_18.to(device)

# Set the model to evaluation mode
resnet_18.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = resnet_18(inputs)
        _, preds = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

resnet18_accuracy = correct / total * 100
print(f'Accuracy of the network on the test images: {resnet18_accuracy:.4f}')


Accuracy of the network on the test images: 64.3194


In [16]:
# Load the model
from transformers import DeiTForImageClassification

model_deit = DeiTForImageClassification.from_pretrained(
    'facebook/deit-base-distilled-patch16-224',
    num_labels=102
)
model_deit.load_state_dict(torch.load('model_deit(2).pth'))

# Move the model to the device
model_deit = model_deit.to(device)

# Set the model to evaluation mode
model_deit.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_deit(inputs)
        _, preds = torch.max(outputs.logits, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

deit_accuracy = correct / total * 100
print(f'Accuracy of the network on the test images: {deit_accuracy:.4f}')

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy of the network on the test images: 80.3708


In [21]:
print("Batch Size: 32", "\n")
print("VGG16 Accuracy: ", vgg16_accuracy)
print("VGG19 Accuracy: ", vgg19_accuracy)
print("ResNet18 Accuracy: ", resnet18_accuracy)
print("DeiT Accuracy: ", deit_accuracy)



Batch Size: 32 

VGG16 Accuracy:  73.23141974304765
VGG19 Accuracy:  71.05220361034314
ResNet18 Accuracy:  64.31940152870385
DeiT Accuracy:  80.37079199869898
